
# 🌡️ ANOVA de Temperaturas entre Ciudades (API Pública Open‑Meteo)

Este notebook descarga temperaturas **diarias** de varias ciudades usando la **API pública de Open‑Meteo**, 
verifica supuestos, ejecuta **ANOVA de un factor**, calcula **tamaños de efecto** y realiza pruebas **post-hoc (Tukey HSD)**.  
También produce visualizaciones con **matplotlib** (sin seaborn).

> Para ejecutar la descarga de datos necesitas conexión a Internet.


In [ ]:

import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
from scipy.stats import shapiro, levene, f_oneway, t
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd


In [ ]:

# --- Configuración de ciudades y rango de fechas ---
cities = {
    "Bogota": (4.61, -74.08),
    "CDMX": (19.43, -99.13),
    "Madrid": (40.42, -3.70),
    "BuenosAires": (-34.60, -58.38),
    "NewYork": (40.71, -74.00),
}

# Rango: últimos 10 días (incluyendo hoy)
end_date = datetime.now().date()
start_date = end_date - timedelta(days=9)

START = start_date.isoformat()
END = end_date.isoformat()
START, END


In [ ]:

# --- Descarga de datos desde Open‑Meteo ---
# Usaremos el endpoint de archive para obtener series horarias y promediar por día.

ARCHIVE_URL = "https://archive-api.open-meteo.com/v1/archive"

def fetch_city_daily_mean_temp(city_name: str, lat: float, lon: float, start: str, end: str) -> pd.DataFrame:
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start,
        "end_date": end,
        "hourly": "temperature_2m",
        "timezone": "auto"
    }
    r = requests.get(ARCHIVE_URL, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    hourly = js.get("hourly", {})
    times = hourly.get("time", [])
    temps = hourly.get("temperature_2m", [])
    if not times or not temps:
        return pd.DataFrame(columns=["city", "date", "temperature"])
    df = pd.DataFrame({"datetime": pd.to_datetime(times), "temperature": temps})
    df["date"] = df["datetime"].dt.date
    daily = df.groupby("date", as_index=False)["temperature"].mean()
    daily["city"] = city_name
    return daily[["city", "date", "temperature"]]


In [ ]:

# --- Descarga y consolidación ---
frames = []
for name, (lat, lon) in cities.items():
    try:
        part = fetch_city_daily_mean_temp(name, lat, lon, START, END)
        frames.append(part)
    except Exception as e:
        print(f"Error con {name}: {e}")

df = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(columns=["city","date","temperature"])
print("Filas descargadas:", len(df))
df.head()


In [ ]:

# --- Limpieza y chequeos básicos ---
df = df.dropna(subset=["temperature"]).copy()
df["city"] = df["city"].astype("category")
df["date"] = pd.to_datetime(df["date"])
print(df.groupby("city")["temperature"].describe())


In [ ]:

# --- Boxplot por ciudad (matplotlib puro) ---
plt.figure(figsize=(8,5))
data_by_city = [df[df["city"] == c]["temperature"].values for c in df["city"].cat.categories]
plt.boxplot(data_by_city, labels=list(df["city"].cat.categories), showmeans=True)
plt.xlabel("Ciudad")
plt.ylabel("Temperatura diaria media (°C)")
plt.title("Distribución de temperaturas por ciudad")
plt.grid(True, axis="y", linestyle="--", alpha=0.6)
plt.show()


In [ ]:

# --- Supuestos ANOVA ---
# Shapiro-Wilk por grupo (nota: con pocas observaciones, el poder es limitado)
normality_results = {}
for c in df["city"].cat.categories:
    vals = df.loc[df["city"] == c, "temperature"].values
    if len(vals) >= 3:
        stat, p = shapiro(vals)
        normality_results[c] = {"n": len(vals), "W": stat, "pvalue": p}
    else:
        normality_results[c] = {"n": len(vals), "W": np.nan, "pvalue": np.nan}

# Levene (homogeneidad de varianzas)
groups = [df.loc[df["city"] == c, "temperature"].values for c in df["city"].cat.categories]
lev_stat, lev_p = levene(*groups)

print("== Shapiro-Wilk por ciudad ==")
for c, res in normality_results.items():
    print(c, res)

print("\n== Levene ==")
print(f"Statistic={lev_stat:.4f}, p-value={lev_p:.4f}")


In [ ]:

# --- ANOVA: scipy (f_oneway) ---
anova_stat, anova_p = f_oneway(*groups)
print(f"ANOVA (scipy) → F={anova_stat:.4f}, p-value={anova_p:.6f}")

# --- ANOVA y tamaños de efecto con statsmodels ---
model = ols("temperature ~ C(city)", data=df).fit()
anova_tbl = sm.stats.anova_lm(model, typ=2)  # incluye sum_sq
anova_tbl


In [ ]:

# --- Tamaños de efecto ---
ss_between = anova_tbl.loc["C(city)", "sum_sq"]
ss_resid = anova_tbl.loc["Residual", "sum_sq"]
sst = ss_between + ss_resid
df_between = anova_tbl.loc["C(city)", "df"]
df_resid = anova_tbl.loc["Residual", "df"]

eta_sq = ss_between / sst if sst > 0 else np.nan
omega_sq = (ss_between - df_between * (ss_resid / df_resid)) / (sst + (ss_resid / df_resid)) if df_resid > 0 else np.nan

print(f"η² (eta squared): {eta_sq:.4f}")
print(f"ω² (omega squared): {omega_sq:.4f}")


In [ ]:

# --- Post-hoc: Tukey HSD ---
tukey = pairwise_tukeyhsd(endog=df["temperature"], groups=df["city"], alpha=0.05)
print(tukey.summary())
# Guardar resultados a CSV
tukey_df = pd.DataFrame(data=tukey._results_table.data[1:], columns=tukey._results_table.data[0])
out_tukey = "/mnt/data/tukey_results.csv"
tukey_df.to_csv(out_tukey, index=False)
print("Tukey guardado en:", out_tukey)


In [ ]:

# --- Medias e IC95% por ciudad ---
summary = df.groupby("city")["temperature"].agg(["mean", "std", "count"]).reset_index()
summary["se"] = summary["std"] / np.sqrt(summary["count"])
# IC con t de Student
summary["tcrit"] = summary["count"].apply(lambda n: t.ppf(1-0.025, df=n-1) if n>1 else np.nan)
summary["ci95"] = summary["tcrit"] * summary["se"]
summary


In [ ]:

# --- Plot de medias con IC95% ---
plt.figure(figsize=(8,5))
x = np.arange(len(summary))
means = summary["mean"].values
ci = summary["ci95"].values

plt.errorbar(x, means, yerr=ci, fmt="o", capsize=5)
plt.xticks(x, summary["city"].tolist(), rotation=0)
plt.ylabel("Temperatura diaria media (°C)")
plt.title("Medias por ciudad con IC95%")
plt.grid(True, axis="y", linestyle="--", alpha=0.6)
plt.show()


In [ ]:

out_csv = "/mnt/data/anova_temperaturas_ciudades.csv"
df.to_csv(out_csv, index=False)
print("Datos guardados en:", out_csv)



## 🧠 Comentarios finales
- **ANOVA de un factor** evalúa si las **medias** de temperatura difieren entre ciudades.
- Se verificaron **supuestos** (normalidad por grupo y homocedasticidad). Con pocas observaciones, interpreta con cautela.
- Se reportan **tamaños de efecto** (η², ω²) para cuantificar magnitud, no solo significancia.
- **Tukey HSD** identifica **qué pares** de ciudades difieren con control del error familiar.
- Si los supuestos no se cumplen, considera alternativas robustas (Kruskal‑Wallis + Dunn).

> Repite el análisis con **más días** o **variables meteorológicas** (humedad, viento) para mayor potencia y contexto.
